In [ ]:
# Sentiment analysis Program of Twitter, Linkdin, Facebook Profile.

In [1]:
#importig librabry
import tweepy
from textblob import TextBlob
import pandas as pd
import numpy as np
import re 
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#loading Data
from google.colab import files
uploaded= files.upload()

In [ ]:
#get the data
log = pd.read_csv('login.csv')

In [ ]:
#Twitter Api Credentials
consumer_key = log['key'][0]
consumer_secret = log['key'][1]
access_token =log[key][2]
access_token_secret = log[key][3]

In [ ]:
#creating the aythentication object
authenticate = tweepy.OAuthHandler(consumer_key, consumer_secret)

#set access token & secret
authenticate.set_access_token(access_token, access_token_secret)

#Create the Api object while passing in the authentication Information
api = tweepy.API(authenticate,wait_on_rate_limit=True)

In [ ]:
#Pulling 100 Tweets from the selected profile name
i=1
post = api.user_timeline(screen_name ='BillGates',count=100,lang='en',tweet_mode='extended')

for tweet in Post[0:5]:
    print( str(i)+') '+ tweet.full_text + '\n')
    i++

In [ ]:
#creating a DataFrame
df=pd.DataFrame([tweet.full_text for tweet in post], columns=['Tweets'])

#show first 5 Text
df.head()

In [ ]:
#Cleaning the Text

#creating a Function to clean the tweets
def cleanText(text):
  text = re.sub(r'@[A-Za-z0-9]+','',text) #removing the @mentions
  text = re.sub(r'#','',text) #removing the "#" symbloe
  text = re.sub(r'RT[\s]+','',text) #removing Retweet
  text = re.sub(r'https?:\/\/\S+','',text) #removing the Hyperlinks

  return text

In [ ]:

#cleaning the text
df['Tweets']= df['Tweets'].apply(cleanText)

#showing the clean tweet
df

In [ ]:
#Creating a function to get the subjectivity 
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

#Creating a function to get the Polarity
def getPolarity(text):
  return TextBlob(text).sentiment.polarity

#Create two new Coloumn Subjectivity & polarity
df['Subjectivity']=df['Tweets'].apply(getSubjectivity)
df['Polarity']=df['Tweets'].apply(getPolarity)

#Showing Data with Subjectivity & polarity
df

In [ ]:
#Creating a Function to Compute Negative(-1), Neutral(0), Positive(+1) Analysis Score

def getAnalysisPolarityScore(score):
  if score<0:
    return 'Negative'
  elif score==0:
    return 'Neutral'
  else:
    return 'Positive'


#Creating a Function to compute Fact(0),Opinion(1)
def getAnalysisSubjectivityScore(score):
  if score <0.5:
    return 'Fact'
  else:
    return 'Opinion'

In [ ]:

df['PolarityAnalysis'] = df['Polarity'].apply(getAnalysisPolarityScore)
df['SubjectivityAnalysis']=df['Subjectivity'].apply(getAnalysisSubjectivityScore)

#showing data
df

In [ ]:
#Printing the Percentage of Positive Tweet with Facts:

pftweets = df[(df.PolarityAnalysis =='Positive') & (df.SubjectivityAnalysis =='Fact')]
pftweets =pftweets['Tweets']

potweets = df[(df.PolarityAnalysis =='Positive') & (df.SubjectivityAnalysis =='Opinion')]
potweets =potweets['Tweets']

nftweets = df[(df.PolarityAnalysis =='Negative') & (df.SubjectivityAnalysis =='Fact')]
nftweets =nftweets['Tweets']

notweets = df[(df.PolarityAnalysis =='Negative') & (df.SubjectivityAnalysis =='Opinion')]
notweets =notweets['Tweets']

nuftweets = df[(df.PolarityAnalysis =='Neutral') & (df.SubjectivityAnalysis =='Fact')]
nuftweets =nuftweets['Tweets']

nuotweets = df[(df.PolarityAnalysis =='Neutral') & (df.SubjectivityAnalysis =='Opinion')]
nuotweets =nuotweets['Tweets']

In [ ]:
# calculating % of Diffrent kind of Tweets:

pf = round ((pftweets.shape[0]/df.shape[0])*100,2)
po = round ((potweets.shape[0]/df.shape[0])*100,2)
nf = round ((nftweets.shape[0]/df.shape[0])*100,2)
no = round ((notweets.shape[0]/df.shape[0])*100,2)
nuf = round ((nuftweets.shape[0]/df.shape[0])*100,2)
nuo = round ((nuoweets.shape[0]/df.shape[0])*100,2)

In [ ]:
#plotting & vusualising the Personality in two differnt Plots

# Plotting Scatter Plot:
plt.figure(figsize=(10,10)) 
for i in range(0, df.shape[0]):
  plt.scatter(df["Polarity"][i], df["Subjectivity"][i], color='Blue') # plt.scatter(x,y,color)   
plt.title('Sentiment Analysis') 
plt.xlabel('Polarity') 
plt.ylabel('Subjectivity') 
plt.show()


# Plotting Radar Plot:

df2 = pd.DataFrame(dict(r =[nf, nuf , pf, po, nuo, no], theta =['Negative Facts', 'Neutral Facts', 'Positive Facts', 'Positive Opinion', 'Neutral Opinion', 'Negative Opinion']))
fig = px.line_polar(df2, r='r',theta='theta',line_close=True)
fig.update_traces(fill='toself')
fig.show()